In [2]:
!pip install numpy pandas tensorflow

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
dataset = pd.read_csv('Exp_Mn_Mw_Value.txt', sep='\t')

In [5]:
X = dataset.iloc[:, 1:5].values   # columns 1–4: Factor A–D
y = dataset.iloc[:, 5:7].values   # columns 5–6: Responses Mn, Mw

In [6]:
dataset.head()

,Run,Factor A,Factor B,Factor C,Factor D,Response 1 (Experimental),Response 2 (Experimental)
0,1,110,7,50,10,1127.19,1321.65
1,2,85,13,50,10,1024.97,1339.35
2,3,101,1,500,60,1950.00,2878.90
3,4,101,1,500,60,2223.17,2989.00
4,5,50,10,50,10,1845.60,2690.50


In [7]:
!pip install scikit-learn

In [8]:
from sklearn.model_selection import train_test_split
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1111, shuffle=True, random_state=42)

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [10]:
from tensorflow.keras import Input

model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='linear'))  # outputs Mn & Mw

In [11]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mape'])

In [12]:
print(X_train.shape)  # (samples, features)
print(y_train.shape)  # (samples, 2)

(17, 4)
(17, 2)


In [13]:
y_pred = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step


In [14]:
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop],
                    shuffle=True,
                    verbose=1)

Epoch 1/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 701ms/step - loss: 9865134.0000 - mape: 99.9927 - val_loss: 7577453.5000 - val_mape: 99.9924
Epoch 2/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 9864874.0000 - mape: 99.9908 - val_loss: 7577221.5000 - val_mape: 99.9910
Epoch 3/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 9864610.0000 - mape: 99.9890 - val_loss: 7576988.0000 - val_mape: 99.9896
Epoch 4/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 9864343.0000 - mape: 99.9871 - val_loss: 7576764.0000 - val_mape: 99.9882
Epoch 5/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 9864073.0000 - mape: 99.9852 - val_loss: 7576536.0000 - val_mape: 99.9868
Epoch 6/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 9863807.0000 - mape: 99.9832 - val_loss: 7576304.0000 - val_mape: 99.9854
Epoch 7/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 9863536.0000 - mape: 99.9813 - val_loss: 7576066.5000 - val_mape: 99.9839
Epoch 8/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 9863263.0000 - mape: 99.9

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
print("MAPE:", mean_absolute_percentage_error(y_test, y_pred))
print("R2 :", r2_score(y_test, y_pred))
comparison = np.hstack((y_test, y_pred))
print("Actual Mn, Actual Mw, Predicted Mn, Predicted Mw:")
print(comparison)

MAPE: 0.9999176556290215
R2 : -6.644866544788956
Actual Mn, Actual Mw, Predicted Mn, Predicted Mw:
[[ 2.29862000e+03  2.97298000e+03  3.90291810e-01  1.68740794e-01]
 [ 3.76288000e+03  5.75281000e+03  3.38216126e-01  1.43372223e-01]
 [ 1.12719000e+03  1.32165000e+03  1.75338671e-01 -1.70701727e-01]
 [ 2.95583000e+03  2.96691000e+03  4.54235733e-01  3.31150949e-01]
 [ 2.32283000e+03  2.98731000e+03  2.97779292e-01  1.85826987e-01]]


In [16]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_percentage_error

# Create and train SVM model
svm_model = MultiOutputRegressor(SVR(kernel='rbf', C=100, gamma='scale', epsilon=0.1))
svm_model.fit(X_train, y_train)

# Predict using SVM
y_pred_svm = svm_model.predict(X_test)

# Evaluate SVM
mape_svm = mean_absolute_percentage_error(y_test, y_pred_svm)
r2_svm = r2_score(y_test, y_pred_svm)

print("\nSVM Results (Mn & Mw):")
print("MAPE:", mape_svm)
print("R² :", r2_svm)

mape_mw = mean_absolute_percentage_error(y_test[:, 1], y_pred_svm[:, 1])
print("MAPE (Mw only):", mape_mw)

# Compare actual vs predicted
comparison_svm = np.hstack((y_test, y_pred_svm))
print("Actual Mn, Actual Mw, Predicted Mn, Predicted Mw (SVM):")
print(comparison_svm)


SVM Results (Mn & Mw):
MAPE: 0.30478485457008747
R² : 0.26304485791909066
MAPE (Mw only): 0.33147024671334063
Actual Mn, Actual Mw, Predicted Mn, Predicted Mw (SVM):
[[2298.62       2972.98       2625.27988953 3215.48294716]
 [3762.88       5752.81       2624.94168802 3214.09948328]
 [1127.19       1321.65       2041.90917578 2734.28148522]
 [2955.83       2966.91       2558.52351422 3135.9898253 ]
 [2322.83       2987.31       2322.9599938  3013.16575989]]


In [17]:
!pip install matplotlib

In [18]:
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from sklearn.metrics import mean_squared_error


In [20]:
transfer_functions = {
    'tanh': 'Tansig',
    'sigmoid': 'Logsig',
    'linear': 'Purelin',
    'relu': 'Radbas'
}

In [21]:
mape_results = []

In [22]:
# Loop over activation functions
from sklearn.metrics import mean_absolute_percentage_error
for activation, name in transfer_functions.items():
    # Build the model
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(64, activation=activation))
    model.add(Dense(64, activation=activation))
    model.add(Dense(2, activation='linear'))  # 2 outputs: Mn & Mw

    # Compile the model
    model.compile(optimizer=Adam(), loss='mse')

    # Train the model
    model.fit(X_train, y_train,
              batch_size=32,
              epochs=25,
              validation_data=(X_val, y_val),
              verbose=0,
              shuffle=True)

    # Predict
    y_pred = model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    mape_results.append(mape)
print(len(list(transfer_functions.values())), len(mape_results))
    # Calculate MSE only for Mw (second column)
mape_mw = mean_absolute_percentage_error(y_test[:, 1], y_pred[:, 1])
mse_results.append(mape_mw)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
4 4


NameError: name 'mse_results' is not defined

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(list(transfer_functions.values()), mse_results, marker='o', linestyle='-', color='blue')
plt.title('Mean MAPE of Mw vs Transfer Function (Keras ANN)')
plt.xlabel('Transfer Function')
plt.ylabel('Mean MAPE of Mw')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Step 16: Import additional libraries and implement SVM comparison  
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import (mean_squared_error, r2_score, mean_absolute_error, 
                           explained_variance_score, mean_squared_log_error, 
                           median_absolute_error)


# SVM Implementation
svm_model = MultiOutputRegressor(SVR(kernel='rbf', C=1.0, gamma='scale'))
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Calculate max error manually for multi-output
def multi_output_max_error(y_true, y_pred):
    return np.max(np.abs(y_true - y_pred))

# SVM Results in your requested format (fixed for multi-output)
svm_results = {
    "Explained Variance": explained_variance_score(y_test, y_pred_svm),
    "Max Error": multi_output_max_error(y_test, y_pred_svm),
   "MAPE": mean_absolute_percentage_error(y_test, y_pred_svm),
"MSE" : mean_squared_error(y_test, y_pred_svm),
    "MSLE": mean_squared_log_error(y_test, y_pred_svm),
    "Median AE": median_absolute_error(y_test, y_pred_svm),
    "R²": r2_score(y_test, y_pred_svm)
}

print("SVM Results:")
for metric, value in svm_results.items():
    print(f"{metric}: {value:.6f}")

In [ ]:
# Show how many samples in each set
print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples") 
print(f"Test set: {X_test.shape[0]} samples")
print(f"Total: {X_train.shape[0] + X_val.shape[0] + X_test.shape[0]} samples")

# Predictions for all sets
print(f"\nTest predictions shape: {y_pred_svm.shape}")
# Show actual vs predicted comparison
comparison = pd.DataFrame({
    'Actual_Mn': y_test[:, 0],
    'Actual_Mw': y_test[:, 1], 
    'Predicted_Mn': y_pred_svm[:, 0],
    'Predicted_Mw': y_pred_svm[:, 1]
})
print("Actual vs Predicted Comparison:")
print(comparison)

In [ ]:
#cross validation
# Import required libraries for cross validation
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer

print("5-FOLD CROSS VALIDATION RESULTS")

# Create 5-fold cross validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Use all data (combine train, val, test for proper CV)
X_all = np.vstack([X_train, X_val, X_test])
y_all = np.vstack([y_train, y_val, y_test])

# Create new SVM model for CV
svm_cv_model = MultiOutputRegressor(SVR(kernel='rbf', C=1.0, gamma='scale'))

# Perform cross validation
cv_r2 = cross_val_score(svm_cv_model, X_all, y_all, cv=kf, scoring='r2')
cv_neg_mse = cross_val_score(svm_cv_model, X_all, y_all, cv=kf, scoring='neg_mean_squared_error')
cv_neg_mape = cross_val_score(svm_cv_model, X_all, y_all, cv=kf, scoring='neg_mean_absolute_percentage_error')

# Convert negative scores to positive
cv_mse = -cv_neg_mse
cv_mape = -cv_neg_mape

print("Cross Validation Results (5 folds):")
print(f"R² Mean: {cv_r2.mean():.4f} (±{cv_r2.std():.4f})")
print(f"MSE Mean: {cv_mse.mean():.4f} (±{cv_mse.std():.4f})")
print(f"MAPE Mean: {cv_mape.mean():.4f} (±{cv_mape.std():.4f})")
print()
print(f"Individual R² scores: {cv_r2.round(4)}")
print(f"Individual MSE scores: {cv_mse.round(4)}")
print(f"Individual MAPE scores: {cv_mape.round(4)}")


In [ ]:
# Train final SVM model on all data for comparison
svm_final = MultiOutputRegressor(SVR(kernel='rbf', C=1.0, gamma='scale'))
svm_final.fit(X_all, y_all)

# Get predictions for all data
y_pred_all = svm_final.predict(X_all)

# Simple comparison
print("Actual vs Predicted Comparison (All 25 samples):")

for i in range(len(y_all)):
    print(f"Sample {i+1:2d}: Actual=[{y_all[i,0]:8.2f}, {y_all[i,1]:8.2f}] | Predicted=[{y_pred_all[i,0]:8.2f}, {y_pred_all[i,1]:8.2f}]")

# Or as DataFrame for better visualization
import pandas as pd
comparison_df = pd.DataFrame({
    'Sample': range(1, len(y_all)+1),
    'Actual_Mn': y_all[:, 0],
    'Actual_Mw': y_all[:, 1],
    'Predicted_Mn': y_pred_all[:, 0],
    'Predicted_Mw': y_pred_all[:, 1],
    'Error_Mn': abs(y_all[:, 0] - y_pred_all[:, 0]),
    'Error_Mw': abs(y_all[:, 1] - y_pred_all[:, 1])
})

print("\nDetailed Comparison Table:")
print(comparison_df.round(2))